 TO KNOW MORE ABUOUT SPACY
 
 CHECK HERE : https://spacy.io/usage/rule-based-matching#matcher 

## NOTE:
    Matching extraction will work independent of SPACES in paragraph.

In [1]:
import spacy

In [2]:
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [83]:
nlp = spacy.load('en_core_web_sm')

In [80]:
pattern = [{"ORTH":"iPhone"}, {"ORTH":"X"}]  #here each dictionary is a 1 token..
#ORTH' - IS EITHER UPPER CASE OR LOWER CASE.
#ORTH--> matching word should be match with exact given word...

In [82]:
matcher = Matcher(nlp.vocab)
matcher.add('IPHONE_PATTERN', None ,pattern)  #'some_name', callbakcs = None, pattern to be matched.

In [31]:
doc1 = nlp("New iPhone X release date leaked")

In [32]:
matches = matcher(doc1)
matches

[(9528407286733565721, 1, 3)]

matcher(doc1) --> returuns a list of tuples [(),()]..

each tuple consists of - match_id, start_index of matched span , end_index of matched span
(match_id - hash value of pattern name)

In [16]:
#printing matched words..
for match_id, start , end in matches:
    matched_span = doc1[start:end]
    print(matched_span.text)

Google I/O


In [33]:
pattern1 = [{'LEMMA':'love','POS':'VERB'}, {'POS':'NOUN'}, {'POS':'ADV', 'OP':'+'}] 
#here we are matching verb form love along with noun word
#try to keep "*" in place of "+" and see the changes..

In [34]:
doc1 = nlp("I loved dogs but now I love cats more rapidly")
matcher = Matcher(nlp.vocab)
matcher.add('LOVED_ONE', None ,pattern1)  #callbakcs = None and 1st argument should be something string..

In [35]:
matches = matcher(doc1)
matches

[(11819021744060373232, 6, 9), (11819021744060373232, 6, 10)]

In [36]:
#printing matched words..
for match_id, start , end in matches:
    matched_span = doc1[start:end]
    print(matched_span.text)

love cats more
love cats more rapidly


# working with REGULAR EXPRESSIONS

In [1]:
import re

In [2]:
text = "my phone number is 1256. oh it's wrong. correct one is 1256789987. ok call me!"

In [10]:
phno = re.finditer('[0-9]{10}', text)   #it returns a string which has 10 digit number..
print(phno)
for i in phno:
    print('Phone number = ',i.group())

Phone number =  1256789987


#### wildcard text

In [10]:
re.findall(r'p.', text)  #returns a list of words that starts with 'p' and make sure we place '....' 4 dots since len('phone')=5

['ph']

In [11]:
re.findall(r'p....', text)

['phone']

In [12]:
re.findall(r'\W$', text)  #returns a list containing that special character, if our text is ended with any special character. 

['!']

#### extracing similar strings

In [13]:
re.findall(r'[^\d]+', text)

['my phone number is ', ". oh it's wrong. correct one is ", '. ok call me!']

In [15]:
re.findall(r'[^\D]+', text)

['1256', '1256789987']

In [11]:
text1 = "you can live-long without me on KGP-Talkies"

In [13]:
re.findall("[\w]+-[\w]+", text1)

['live-long', 'KGP-Talkies']

# CALL BACK METHOD IN Matcher.add()

In [42]:
text = "Google annonced a new Pixel at Google I/O . Google I/O is great place to get all updtes from Google."

in the above text, should have a space between "Google I/O" and "."

In [43]:
pattern = [{'TEXT':'Google'}, {'TEXT':'I'}, {'TEXT':'/'}, {'TEXT':'O'}]

In [44]:
def callback_method(matcher, doc, i, matches): 
    print(i)        #prints the starting index of matches.
    match_id, start, end = matches[i]
    entity = doc[start:end]
    print(entity.text)

###### callback_method()
this method executes when we run the "matcher(doc)" then pointer returns to callback_method() as inputs -
 1)matcher object(pattern), 2)doc string, 3)firstly "i" pointing to zeroth location of matched tuples, 4)list of matched tuples.
##### NOTE:
     callback_method() executed by Matcher object automatically call until indexes(returned) are empty.

In [45]:
matcher = Matcher(nlp.vocab)
matcher.add('Google', callback_method, pattern)


In [46]:
doc = nlp(text)

In [47]:
matcher(doc)


0
Google I/O
1
Google I/O


[(11578853341595296054, 6, 10), (11578853341595296054, 11, 15)]

#### we can also do this task with regular expression

In [39]:
import re
re.findall("Google I/O", text)

['Google I/O', 'Google I/O']

# Text extration using Lingustic analysis

Let's say if you are analysing user comments and you want to find out what people are saying about Facebook.
You  want to start by finding sentences containing following as'Facebook is' and 'Facebook was'. so. lets begin...

In [4]:
matcher = Matcher(nlp.vocab)

In [14]:
matched_sents = []

In [15]:
pattern = [{"LOWER":"facebook"}, {"LEMMA":"be"}, {"POS":"ADV","OP":"*"}, {"POS": "ADJ"}]

In [16]:
def callback_method_fb(matcher, doc, i, matches):
    matched_id, start, end = matches[i]
    span = doc[start:end]
    sent = span.sent    #it returns a  corresponding sentence in which "span" is present
    
    
    match_ents = [{
        
        "start": span.start_char - sent.start_char,
        'end': span.end_char - sent.start_char,
        'label': 'MATCH'
    }]
    
    matched_sents.append({'text': sent.text, 'ents': match_ents})
    

In [17]:
matcher.add("fb", callback_method_fb, pattern)

In [18]:
doc = nlp("I'd say that Facebook is evil. - Facebook is pretty cool, right?")


In [19]:
matches = matcher(doc)
matches

[(8017838677478259815, 4, 7), (8017838677478259815, 9, 13)]

In [20]:
matched_sents

[{'text': "I'd say that Facebook is evil.",
  'ents': [{'start': 13, 'end': 29, 'label': 'MATCH'}]},
 {'text': 'Facebook is pretty cool, right?',
  'ents': [{'start': 0, 'end': 23, 'label': 'MATCH'}]}]

In [21]:
#visulalising data

In [22]:
displacy.render(matched_sents, style = 'ent', manual = True)
#manual (True): it tells us that Don't parse `Doc` instead we expect a dict/list of dicts.

## Phone numbers extraction

phone numbers will have different format in different countries. so your matched pattern will have to look for number sequences 
satistys required condition.. 

In [28]:
pattern = [{"ORTH": "("}, {"SHAPE": "ddd"}, {"ORTH": ")"}, {"SHAPE": "dddd"}, {"ORTH": "-", "OP":"?"}, {"SHAPE": "dddd"}]

In [29]:
matcher = Matcher(nlp.vocab)
matcher.add("PhoneNumber", None, pattern)

In [30]:
doc = nlp("Call me at (123) 4560 7890")

In [31]:
print([t.text for t in doc])

['Call', 'me', 'at', '(', '123', ')', '4560', '7890']


In [32]:
matches = matcher(doc)
matches

[(7978097794922043545, 3, 8)]

In [33]:
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

(123) 4560 7890


### Email Address matching using REGEX in Spacy

In [59]:
pattern = [{"TEXT": {"REGEX": "[a-z0-9-_.]+@[a-z0-9.]+"}}]  #here "+" sign indicates to add all characters matched.
#NOTE: there shouldn't be any space between {"REGEX": ""}

In [60]:
matcher = Matcher(nlp.vocab)
matcher.add("Email", None, pattern)

In [61]:
text = "Email me at kartheeknella2@gmail.com and nellakartheek2000@gmail.com"

In [62]:
doc = nlp(text)

In [63]:
print([token.text for token in doc])

['Email', 'me', 'at', 'kartheeknella2@gmail.com', 'and', 'nellakartheek2000@gmail.com']


In [56]:
matches = matcher(doc)
matches

[(11010771136823990775, 3, 4), (11010771136823990775, 5, 6)]

In [64]:
for match_id, start,end in matches:
    span = doc[start:end]
    print(span)

kartheeknella2@gmail.com
nellakartheek2000@gmail.com


# Hashtags and emoji detection in social media

In [48]:
pos_emoji = ["😃","😋","🥰","😘","😃","🤗"] 
neg_emoji = ["😣","😔","☹️","😬","😩","😡"]  #make sure that no space is given between "".

In [49]:
pos_patterns = [[{"ORTH": emoji}] for emoji in pos_emoji]
neg_patterns = [[{"ORTH": emoji}] for emoji in neg_emoji]

In [50]:
pos_patterns

[[{'ORTH': '😃'}],
 [{'ORTH': '😋'}],
 [{'ORTH': '🥰'}],
 [{'ORTH': '😘'}],
 [{'ORTH': '😃'}],
 [{'ORTH': '🤗'}]]

In [51]:
def label_sentiment(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    if doc.vocab.strings[match_id] == "HAPPY":
        doc.sentiment += 0.1       #initially doc.sentiment = 0.0 .
    elif doc.vab.string[match_id] == 'SAD':
        doc.sentiment -= 0.1
   

In [52]:
matcher = Matcher(nlp.vocab)
matcher.add("HAPPY", label_sentiment, *pos_patterns)  #if we give a (more than one pattern)list of patterns,
                                                              #then syntax is -- *Patterens
matcher.add("SAD", label_sentiment, *neg_patterns)

In [53]:
matcher.add("HASHTAG", None, [{"TEXT":"#"}, {"IS_ASCII": True}]) #'IS_ASCII' -- is used to receive a name with no white space

In [54]:
doc = nlp("Hello World 😃 #NELLAKartheek")

In [55]:
print([token.text for token in doc])

['Hello', 'World', '😃', '#', 'NELLAKartheek']


In [56]:
matches = matcher(doc)
matches

[(2686646543460454932, 2, 3), (16536914698459818706, 3, 5)]

#### NOTE:
        if spacy having in-built objects or functions then those can be used directly outside the callback method by without              returning those objects or functions in call back method.
           below line is example:

In [57]:
print(doc.sentiment)

0.10000000149011612


In [115]:
for match_id, start, end in matches:
    string_id = doc.vocab.strings[match_id]
    span = doc[start:end]
    print(f'{string_id:10}', span.text)

HAPPY      😃
HASHTAG    #NELLAKartheek


# Efficient phrase matching

 if you need to match large terminology lists,then you can also use the PhraseMatcher and creates Doc objects instead of token patterns, which is much more efficient overall. The Doc patterns can contain single or multile tokens.

In [68]:
from spacy.matcher import PhraseMatcher

In [69]:
matcher= PhraseMatcher(nlp.vocab)

In [70]:
terms = ['BARAC OBAMA', 'ANGELA MERKEL', 'WASHINGTON D.C']

In [71]:
pattern = [nlp.make_doc(text) for text in terms]   #make_doc -- it makes text string into doc
pattern

[BARAC OBAMA, ANGELA MERKEL, WASHINGTON D.C]

In [73]:
matcher.add('term', None, *pattern)

In [74]:
doc = nlp("German Chancellor ANGELA MERKEL and US Predident BARAC OBAMA "
         "converse in the Oval Office inside the white House in WASHINGTON D.C")

In [75]:
print([token.text for token in doc])

['German', 'Chancellor', 'ANGELA', 'MERKEL', 'and', 'US', 'Predident', 'BARAC', 'OBAMA', 'converse', 'in', 'the', 'Oval', 'Office', 'inside', 'the', 'white', 'House', 'in', 'WASHINGTON', 'D.C']


In [76]:
doc

German Chancellor ANGELA MERKEL and US Predident BARAC OBAMA converse in the Oval Office inside the white House in WASHINGTON D.C

In [77]:
matches = matcher(doc)
matches

[(4519742297340331040, 2, 4),
 (4519742297340331040, 7, 9),
 (4519742297340331040, 19, 21)]

In [78]:
for match_id, start,end in matches:
    span = doc[start:end]
    print(span.text)

ANGELA MERKEL
BARAC OBAMA
WASHINGTON D.C


# Custom Rule Based Entity Recognition

to add named entities based on pattern dictionaries and makes it easy to combine rule-based and statistical named entity recognition for even more powerful models.

#### Entity Patterns

#### working of entity ruler

CHECK HERE : https://spacy.io/api/annotation#named-entities

In [5]:
from spacy.pipeline import EntityRuler

In [42]:
nlp = spacy.load('en_core_web_sm')

In [43]:
ruler = EntityRuler(nlp)

In [44]:
patterns = [{"label":"ORG","pattern": "KGPTalkie"}, {"label": "GPE", "pattern":[{"LOWER": "san"}, {"LOWER": "francisco"}]}]

In [45]:
ruler.add_patterns(patterns)

In [46]:
nlp.add_pipe(ruler)

In [84]:
doc = nlp("KGPTalkie is opening its first big office in San Francisco.")

In [85]:
print([token.text for token in doc])

['KGPTalkie', 'is', 'opening', 'its', 'first', 'big', 'office', 'in', 'San', 'Francisco', '.']


In [86]:
for ent in doc.ents:
    print(f'{ent.text:20}', ent.label_)

KGPTalkie            ORG
first                ORDINAL
San Francisco        GPE


### NOTE: 
  TO GET the explanation about terms in named entity regonition, then use "spacy.explain("ORG")"

In [89]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'